<h1>Предсказание успешности проектов, собирающих средства на Kickstarter при помощи моделей МО</h1>
<h2>Описание задачи</h2>

<p>Выбранный набор данных представляет из себя статистику всех краудфандинговых кампаний, которые были запущены на платформе Kickstarter с 2009 по 2018 годы. Он включает в себя информацию о проектах, таких как название, категория, дата запуска и дедлайн, сумма, которую он собрал, сумма, которую планировалось собрать и прочие метрики. 

На основании этих данных можно обучить разные модели, которые смогут предсказывать успешность проекта на основе параметров его кампании на Kickstarter. В качестве меры успешности кампании будет выступать поле "state" со следующими значениями:</p>

In [ ]:
import pandas as pd 
df = pd.read_csv('dataset.csv')
print(df['state'].unique())

['failed' 'canceled' 'successful' 'live' 'undefined' 'suspended']


<p>Состояния live и undefined нам особо не интересны; suspended, canceled и failed свидетельствуют о провале кампании. Значит, при обучении модели успешными стоит считать только записи со статусом successful. Таким образом, приняв state = 'successful' за 1, а все остальные значения за 0, мы можем свести задачу предсказания успешности кампании к классической задаче классификации, которую можно решить методом логистической регрессии.</p>

<h2>Анализ, первичная чистка и обработка данных</h2>
<p>Датасет содержит большое количество данных, не все из которых имеют значение в рамках поставленной задачи. 
Выведем некоторую базовую информацию о наборе данных в исходном состоянии</p>

In [1]:

import pandas as pd 

df = pd.read_csv('dataset.csv')

print("Data Information :\n ")
print(df.info())

print("Number of nulls for column : \n")
print(df.isnull().sum())

print("Data statistics : \n")
print(df.describe())


Data Information :
 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 378661 entries, 0 to 378660
Data columns (total 15 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   ID                378661 non-null  int64  
 1   name              378657 non-null  object 
 2   category          378661 non-null  object 
 3   main_category     378661 non-null  object 
 4   currency          378661 non-null  object 
 5   deadline          378661 non-null  object 
 6   goal              378661 non-null  float64
 7   launched          378661 non-null  object 
 8   pledged           378661 non-null  float64
 9   state             378661 non-null  object 
 10  backers           378661 non-null  int64  
 11  country           378661 non-null  object 
 12  usd pledged       374864 non-null  float64
 13  usd_pledged_real  378661 non-null  float64
 14  usd_goal_real     378661 non-null  float64
dtypes: float64(5), int64(2), object(8)
memory usage

Из вывода видно, что в таблице аж 378 тысяч строк и почти нет пустых (null) значений. Это позволяет отбросить null-значения и всё равно оставить более чем достаточное количество данных для обучения и тестирования моделей.
Так же статистика показывает, что среднее значение (mean) поля usd_pledged_real (кол-во инвестированных денег в переводе в доллары США) заметно меньше значения usd_goal_real (цель кампании по сбору денег в долларах США). Из этого можно сделать вывод, что в среднем краудфандинговые кампании не достигают поставленных целей и стоит ожидать больше отрицательных результатов, чем положительных.

Рассматривая столбцы по отдельности можно выделить заведомо не имеющие значения для нас столбцы, а также сделать предположения о важности тех или иных признаков, которые мы эмпирически проверим позже. Так, например, столбец ID очевидно не несёт полезной информации, usd_pledged отличается от usd_pledged_real лишь инструментом конвертации суммы в USD и по своему смыслу полностью дублирует его, а goal и pledged не только дублируются, но и вообще выражены в изначальной валюте сбора и, соответственно, не нормализованы относительно строк с другими показателями валюты. Столбцы currency (валюта) и country (страна) вероятно также не имеют особого значения, но они всё же несут уникальную информацию, релевантность которой стоит проверить.

Смотря от обратного, можно заметить, что name, category и main_category можно использовать для категоризации, backers (кол-во поддержавших), usd_goal_real, usd_pledged_real имеют непосредственное отношение к финансовому аспекту, а launched и deadline - к сроку кампании. Исходя из этого можно предположить, что значения в этих столбцах будут иметь значение для прогнозирования успеха кампании.

Тем не менее, для более удобного взаимодействия с этими данными их стоит обработать. А именно:
-Конвертировать строковые значения статуса (state) в булевы значения успешности (success)
-Конвертировать даты в формат datetime, чтобы с ними можно было действительно работать как с датами при помощи pandas
-Объединить launched и deadline в одно поле, указывающее на продолжительность кампании в одном значении
-Создать дополнительные признаки, касающиеся времени старта кампании

Для подготовки к обучению моделей также стоит закодировать категориальные признаки и выделить параметры (фичи)

In [ ]:
def preprocess_data(df):
    """
    Предобработка данных Kickstarter
    """
    # Создаем копию данных
    data = df.copy()
    
    # Создаем целевую переменную (1 - успешная, 0 - неуспешная)
    data['success'] = (data['state'] == 'successful').astype(int)
    
    # Удаляем строки с отсутствующими значениями в ключевых столбцах
    data = data.dropna(subset=['usd_goal_real', 'usd_pledged_real', 'backers'])
    
    # Обработка дат
    data['launched'] = pd.to_datetime(data['launched'])
    data['deadline'] = pd.to_datetime(data['deadline'])
    data['campaign_duration'] = (data['deadline'] - data['launched']).dt.days
    
    # Создание дополнительных признаков
    data['launch_year'] = data['launched'].dt.year
    data['launch_month'] = data['launched'].dt.month
    data['launch_day_of_week'] = data['launched'].dt.dayofweek
    
    # Категориальные переменные
    label_encoders = {}
    categorical_columns = ['category', 'main_category', 'currency', 'country']
    
    for col in categorical_columns:
        if col in data.columns:
            le = LabelEncoder()
            data[col + '_encoded'] = le.fit_transform(data[col].astype(str))
            label_encoders[col] = le
    
    # Выбираем признаки для модели
    feature_columns = [
        'usd_goal_real', 'backers', 'campaign_duration',
        'launch_year', 'launch_month', 'launch_day_of_week'
    ]
    
    # Добавляем закодированные категориальные признаки
    for col in categorical_columns:
        if col + '_encoded' in data.columns:
            feature_columns.append(col + '_encoded')
    
    # Удаляем выбросы (используем IQR метод для числовых столбцов)
    numeric_features = ['usd_goal_real', 'backers', 'campaign_duration']
    for col in numeric_features:
        if col in data.columns:
            Q1 = data[col].quantile(0.25)
            Q3 = data[col].quantile(0.75)
            IQR = Q3 - Q1
            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR
            data = data[(data[col] >= lower_bound) & (data[col] <= upper_bound)]
    
    return data, feature_columns, label_encoders